In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dask.dataframe as dd


import os
from glob import glob

In [2]:
os.chdir('/scratch/work/courses/DSGA1004-2021/movielens/ml-latest-small')

In [3]:
movies_df = dd.read_csv('movies.csv')
movies_df = movies_df.repartition(npartitions=500)

ratings_df = dd.read_csv('ratings.csv')
#ratings_df = ratings_df.repartition(npartitions=ratings_df.npartitions // 100)

links_df = dd.read_csv('links.csv')
#links_df = links_df.repartition(npartitions=links_df.npartitions // 100)

tags_df = dd.read_csv('tags.csv')
#tags_df = tags_df.repartition(npartitions=tags_df.npartitions // 100)

In [4]:
print("This dataset contains {} rows and {} columns".format(len(movies_df), movies_df.shape[1]))
movies_df.head()

This dataset contains 9742 rows and 3 columns


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
links_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [7]:
tags_df.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [8]:
%%time
ratings_df['movieId'] = ratings_df['movieId'].astype('category').cat.as_known().cat.as_ordered()
ratings_df['userId'] = ratings_df['userId'].astype('category').cat.as_known().cat.as_ordered()
utility_matrix = dd.pivot_table(ratings_df, values='rating', index='userId', columns='movieId')
#utility_matrix.head()

CPU times: user 1.97 s, sys: 18.1 ms, total: 1.99 s
Wall time: 2 s


In [15]:
utility_matrix.columns = utility_matrix.columns.as_ordered()

In [20]:
%%time
#utility_matrix['movieId'] = utility_matrix['movieId'].astype('category').cat.as_known().cat.as_ordered()
#utility_matrix.mean(axis=0).compute()
utility_matrix.mean(axis=0).compute()

CPU times: user 1min 58s, sys: 24.6 s, total: 2min 22s
Wall time: 2min 23s


movieId
1         3.920930
2         3.431818
3         3.259615
4         2.357143
5         3.071429
            ...   
193581    4.000000
193583    3.500000
193585    3.500000
193587    3.500000
193609    4.000000
Length: 9724, dtype: float64